## Домашнее задание 6

В данном домашнем задании Вам предстоит реализовать автоматическое исправление опечаток в запросах пользователей. 

### 1. Датасет
Для оценки качества алгоритма исправления опечаток, Вам предоставляется файл `queries.tsv.gz`. В каждой строке файла записаны два запроса – исходный и исправленный. Для простоты, оба запроса будут иметь одинаковое количество слов и отличаться незначительно. Зачастую исходный и исправленный запрос совпадают, что означает что исправлять такой запрос не требуется.

In [71]:
!~/.pyenv/versions/3.8.5/bin/pip3.8 install tqdm termcolor


In [72]:
from typing import List, Tuple, Generator, Callable

Query = str
Sentence = str
Filename = str
Word = str
Queries = List[Tuple[Query, Query]]


In [73]:
from termcolor import colored
import difflib

def diff_queries(original: Query, fixed: Query) -> Query:
    result = ''
    for pos, d in enumerate(difflib.ndiff(original, fixed)):
        if d[0] == '+':
            result += colored(d[2], 'green')
        elif d[0] == '-':
            result += colored(d[2], 'red')
        else:
            result += d[2]
    return result

print(diff_queries("lake compond the park", "lake compound the park"))
print(diff_queries("traditional chothes", "traditional clothes"))
print(diff_queries("jack sparrow", "captain jack sparrow"))


lake compound the park
traditional chlothes
captain jack sparrow


In [74]:
import gzip

def load_queries(fn: Filename) -> Queries:
    result = []
    with gzip.open(fn, 'rt', encoding='utf8') as inp:
        for line in inp:
            original, fixed = line.rstrip('\n').split('\t')
            result.append((original, fixed))
    return result

queries = load_queries("./queries.tsv.gz")
print(f'Loaded {len(queries)} queries\n')
for original, fixed in queries[10:20]:
    print(diff_queries(original, fixed))


Loaded 102436 queries

embearirassing red carpet moments
grants for rural areas florirda
the home hdepot merchandising
delaware motorcycle inspection requirements
highland park hospital gastric biypass surgery
grand theift auto
windward community college
my credit reports
strack intermediate school
mongol empire political system


In [75]:
queries_sample = [
    ("grand theift auto", "grand theft auto"),
    ("belarus longitude and latitdue", "belarus longitude and latitude"),
    ("search for poeoms", "search for poems"),
    ("large guacolmoi dip restaurtant price", "large guacamole dip restaurant price"),
    ("texas chainsaw mascurer", "texas chainsaw massacre"),
    ("royal trump subtitle", "royal tramp subtitle"),
    ("florida fiberglass polls", "florida fiberglass pools"),
    ("how to make a calender", "how to make a calendar"),
    ("university of south caroline", "university of south carolina"),
    ("maureen mcdonald in virginia", "maureen mcdonnell in virginia"),
]


Для составления словаря и обучения языковых моделей Вам предоставляется небольшой корпус текста, неслучайная выборка из большой английской википедии в файле `train.bz2`. Этот файл содержит примерно 5 млн строк или 80 млн слов. Каждая строка – одно предложение без знаков препинания.
Использование других словарей и корпусов запрещено.

In [76]:
import bz2
from tqdm.notebook import tqdm

def read_huge_corpus(fn: Filename) -> Generator[Sentence, None, None]:
    with bz2.open(fn, 'rt', encoding='utf8') as inp:
        for line in tqdm(inp):
            yield line.rstrip('\n')

for li, line in enumerate(read_huge_corpus("./train.bz2")):
    print(line)
    if li == 10:
        break


gol neshin
mitochondrial dna depletion syndrome mds or mdds is any of a group of autosomal recessive disorders that cause a significant drop in mitochondrial dna in affected tissues
following the relegation of sc freiburg in 2005 he was on the verge of signing for metalurg donetsk but instead he accepted a contract with vfl wolfsburg
the first issue for geometers is what kind of geometry is adequate for a novel situation
cedar grove was formerly a stage and freight stop
regular bus service runs from bhubaneswar to niali which is away
later they were also known for the cream wafer biscuits
strabomantis cornutus
gtk+ scene graph kit gsk was initially released as part of gtk+ 3.90 in march 2017 and is meant for gtk-based applications that wish to replace clutter for their ui
the match took place on 10 april 1906 at the hipódromo madrid
the brothers came from fresno california



### 2. Поиск близких слов
Требуется научится быстро находить список из сотни слов, которые незначительно отличаются от заданного слова.

Не стоит перебирать все слова словаря – займёт слишком много времени.

Для ускорения перебора предлагается создать триграммный индекс – для каждой буквенной триграммы храним список слов, в которых она есть. Тогда для поиска похожих на данное слово найдем слова большим количеством совпадающих триграмм. 

Совет 1: стоит сделать отельный индекс для каждой длинны слова и использовать только те индексы, в которых лежат слова близкие по длине к исходному.

Совет 2: для выделения триграмм стоит обрамить слово спецсимволом, чтобы триграммы на концах слова отличались от оных в середине.

Любые другие алгоритмы, улучшающие качество за разумное время (хождение по бору с ошибками, перебор ошибок) – не возбраняются.

Не побрезгуйте кешировать результат работы этого алгоритма, чтобы дальнейшая работа протекала быстрее.

In [77]:
import pickle
from nltk.tokenize import word_tokenize
import os.path


In [78]:
from collections import defaultdict

if os.path.exists("all_words"):
    all_words = pickle.load(open("all_words", 'rb'))
else:
    all_words = defaultdict(int)
    train = read_huge_corpus("./train.bz2")
    for ind, text in enumerate(train):
        for word in text.split(" "):
            all_words[word] += 1
    all_words = {i: all_words[i] for i in all_words if all_words[i] > 5}
    pickle.dump(all_words, open("all_words", 'wb'))


In [79]:
from collections import defaultdict

if os.path.exists("trigram_index"):
    trigram_index = pickle.load(open("trigram_index", 'rb'))
else:
    trigram_index = defaultdict(set)

    for word in tqdm(all_words):
        temp = "$$" + word + "$$"
        for trigram in [temp[i:i+3] for i in range(len(temp)-2)]:
            trigram_index[(len(word), trigram)].add(word)
    
    for l, t in trigram_index:
        trigram_index[(l, t)] = list(trigram_index[(l, t)])

    pickle.dump(trigram_index, open("trigram_index", 'wb'))


In [80]:
if os.path.exists("words_to_fix"):
    words_to_fix = pickle.load(open("words_to_fix", 'rb'))
else:
    words_to_fix = []

    for query in tqdm(queries):
        for original, fixed in zip(query[0].split(" "), query[1].split(" ")):
            if original != fixed:
                words_to_fix.append((original, fixed))
                
    pickle.dump(words_to_fix, open("words_to_fix", 'wb'))


In [81]:
from collections import Counter

similar_words_cache = {}

def find_similar_words(word: Word, len_gap=2, similar_count=1000) -> List[Word]:
    if word in similar_words_cache:
        return similar_words_cache[word][:similar_count]
    temp = "$$" + word + "$$"
    similar = []
    for trigram in [temp[i:i+3] for i in range(len(temp)-2)]:
        for word_len in range(len(word) - len_gap, len(word) + len_gap + 1):
            if (word_len, trigram) in trigram_index:
                similar += trigram_index[(word_len, trigram)]
    similar = Counter(similar)
    similar_words_cache[word] = sorted(list(similar.keys()), key=lambda w:similar[w] / len(word), reverse=True)
    return similar_words_cache[word][:similar_count]


for original, fixed in words_to_fix[:5]:
    similar = find_similar_words(original)
    print(original, '- ok' if fixed in similar else '- fail')
    for word in similar[:5]:
        print(' ', word)
    print()


chothes - ok
  clothes
  choses
  chores
  chokes
  choices

cataloges - ok
  catalogues
  catalogs
  cataloged
  catalogus
  catalog

compond - ok
  compound
  composed
  component
  compost
  commend

barns - ok
  barns
  barnes
  bairns
  barnas
  barons

emberissing - ok
  embossing
  embarrassing
  embedding
  embezzling
  embellishing



Чтобы оценить качество полученного алгоритма, используйте запросы из `queries.tsv.gz`. Отберите только отличающиеся слова в исправленном и исходном запросах. Проверьте, что для слова в исходном запросе, исправленное слово будет в списке ближайших выданном вашим алгоритмом. Если это выполняется для всех или почти всех пар – успех. 

In [82]:
def extract_different_words(queries: Queries) -> List[Tuple[Word, Word]]:
    words_to_fix = []
    for original, fixed in queries:
        if original != fixed:
            for word_orig, word_fixed in zip(original.split(), fixed.split()):
                if word_orig != word_fixed:
                    words_to_fix.append((word_orig, word_fixed))
    return words_to_fix
                    
words_to_fix = extract_different_words(queries)
print(f'Found {len(words_to_fix)} words to fix')
for original, fixed in words_to_fix[:10]:
    print(diff_queries(original, fixed))


Found 53495 words to fix
chlothes
cataloges
compound
barnes
embearirassing
florirda
hdepot
inspection
biypass
theift


In [83]:
word_to_similar = {}

def check_find_similar_words(words_to_fix: List[Tuple[Word, Word]], 
                             find_similar_words: Callable[[Word], List[Word]], 
                             debug: bool):
    wrong, total = 0, 0
    progress = tqdm(words_to_fix)
    debug_output = 0
    for word_orig, word_fixed in progress:
        similar = find_similar_words(word_orig)
        if word_fixed not in similar:
            wrong += 1
            if debug:
                print(word_orig, word_fixed)
                debug_output += 1
                if debug_output == 10:
                    break
        total += 1
        progress.set_description(f'Wrong: {wrong} - {wrong/total*100:0.2f}%')
        
check_find_similar_words(words_to_fix, find_similar_words, debug=False)


## 3. Языковая модель
Языковая модель – модель, которая по тексту оценивает вероятность того, что он мог появиться в языке. 

Постройте простую n-грамную языковую модель с использованием корпуса текстов `train.bz2`. Для этого рассчитайте количество вхождений каждой n-граммы в корпус текста. Если взять n=2, то размера оперативной памяти вашего компьютера должно будет хватить.

Воспользуйтесь каким-нибудь методом сглаживания, чтобы не получать нулевую вероятность для неизвестных n-грамм. Также, чтобы вероятности слов, которых нет в словаре, были отличны от нуля, можно примешать побуквенную m-граммную модель.

Совет N: если количество оперативной памяти прижмёт, можно хранить строки в виде байт – один раскодированный символ занимает больше памяти чем один байт, при этом для английского текста почти всегда один символ кодируется одним байтом.

In [84]:
n = 2

# word n-gram
if os.path.exists("n_grams"):
    n_grams = pickle.load(open("n_grams", 'rb'))
else:
    n_grams = {}
    train = read_huge_corpus("./train.bz2")
    for ind, text in enumerate(train):
        words = text.split(" ")
        for n_gram in [" ".join(words[i:i+n]) for i in range(len(words)-n+1)]:
            if n_gram not in n_grams:
                n_grams[n_gram] = 0
            n_grams[n_gram] += 1
    pickle.dump(n_grams, open("n_grams", 'wb'))

total_n_grams = len(n_grams) + sum(n_grams[n_gram] for n_gram in n_grams)


In [85]:
m = 3

# char m-gram
if os.path.exists("m_grams"):
    m_grams = pickle.load(open("m_grams", 'rb'))
else:
    m_grams = {}
    train = read_huge_corpus("./train.bz2")
    for ind, text in enumerate(train):
        for word in text.split(" "):
            temp = "$" + word + "$"
            for m_gram in [temp[i:i+m] for i in range(len(temp)-m+1)]:
                if m_gram not in m_grams:
                    m_grams[m_gram] = 0
                m_grams[m_gram] += 1
    pickle.dump(m_grams, open("m_grams", 'wb'))

total_m_grams = len(m_grams) + sum(m_grams[m_gram] for m_gram in m_grams)


In [86]:
from math import log2

total_words = len(all_words) + sum(all_words[word] for word in all_words)

def get_probability(query: Query) -> float: # log probability
    probability = 0
    words = query.split(" ")
    for n_gram in [" ".join(words[i:i+n]) for i in range(len(words)-n+1)]:
        if n_gram in n_grams:
            probability += 0#log2((n_grams[n_gram] + 1) / total_n_grams)
        else:
            probability += -1 * len(words)
            for word in n_gram.split(" "):
                if word in all_words:
                    probability += 0#log2((all_words[word] + 1) / total_words)
                else:
                    probability += -1
                    temp = "$" + word + "$"
                    for m_gram in [temp[i:i+m] for i in range(len(temp)-m+1)]:
                        if m_gram in m_grams:
                            probability += log2((m_grams[m_gram] + 1) / total_m_grams)
    return probability

for original, fixed in queries_sample:
    p_original = get_probability(original)
    p_fixed = get_probability(fixed)
    verdict = '[ok]  ' if p_fixed > p_original else '[fail]'
    sign = '< ' if p_fixed > p_original else '>='
    print(f'{verdict} {original:>40s} {p_original:5.2f}  {sign} {p_fixed:5.2f} {fixed}')


[ok]                          grand theift auto -141.77  <   0.00 grand theft auto
[ok]             belarus longitude and latitdue -118.81  <  -4.00 belarus longitude and latitude
[ok]                          search for poeoms -90.85  <   0.00 search for poems
[ok]      large guacolmoi dip restaurtant price -521.54  <  -130.66 large guacamole dip restaurant price
[ok]                    texas chainsaw mascurer -96.22  <   0.00 texas chainsaw massacre
[ok]                       royal trump subtitle -6.00  <  -3.00 royal tramp subtitle
[fail]                 florida fiberglass polls -6.00  >= -6.00 florida fiberglass pools
[ok]                     how to make a calender -85.75  <   0.00 how to make a calendar
[ok]               university of south caroline -4.00  <   0.00 university of south carolina
[fail]             maureen mcdonald in virginia  0.00  >= -4.00 maureen mcdonnell in virginia


Чтобы оценить качество полученной модели, используйте запросы из `queries.tsv.gz`. Сравните вероятность, которую выдает ваша модель для исходных и исправленных запросов. Хорошая модель выдаёт исправленному запросу большую вероятность. 

In [87]:
def check_language_model(queries: Queries, get_probability: Callable[[Query], float], debug: bool):
    wrong, total = 0, 0
    progress = tqdm(queries)
    debug_output = 0
    for original, fixed in progress:
        if original == fixed:
            continue
        p_original = get_probability(original)
        p_fixed = get_probability(fixed)
        if p_fixed <= p_original:
            wrong += 1
            if debug:
                print(original, p_original)
                print(fixed, p_fixed)
                print()
                debug_output += 1
                if debug_output == 10:
                    break
        total += 1
        progress.set_description(f'Wrong: {wrong} - {wrong/total*100:0.2f}%')
        
check_language_model(queries, get_probability, debug=False)


Советую сохранить полученную модель на диск – а случае чего, чтение статистик с диска, может быть быстрее расчёта оных с нуля.

### 4. Модель ошибок
Модель ошибок – модель которая по исходному и исправленному запросу оценивает вероятность того, что такая ошибка могла быть допущена.

Рассчитайте простую модель ошибок на основе расстояния Дамерау-Левенштейна, то есть модифицированного Левенштейна, который считает перестановку соседних букв за одну ошибку.

In [88]:
def damerau_levenshtein_distance(s1, s2):
    d = {}
    lenstr1 = len(s1)
    lenstr2 = len(s2)
    for i in range(-1,lenstr1+1):
        d[(i,-1)] = i+1
    for j in range(-1,lenstr2+1):
        d[(-1,j)] = j+1

    for i in range(lenstr1):
        for j in range(lenstr2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = 1
            d[(i,j)] = min(
                           d[(i-1,j)] + 1, # deletion
                           d[(i,j-1)] + 1, # insertion
                           d[(i-1,j-1)] + cost, # substitution
                          )
            if i and j and s1[i]==s2[j-1] and s1[i-1] == s2[j]:
                d[(i,j)] = min (d[(i,j)], d[i-2,j-2] + cost) # transposition

    return d[lenstr1-1,lenstr2-1]


In [89]:
from math import log2

def get_error_probability(original: Query, fixed: Query) -> float:
    dist = damerau_levenshtein_distance(original, fixed)
    avg_len = (len(original) + len(fixed)) / 2.
    if avg_len - dist < 0:
        return -1 * 10**10
    return log2((avg_len - dist + 0.00001) / avg_len)

for original, fixed in queries_sample:
    p_error = get_error_probability(original, fixed)
    print(f'{original:>40s} | {p_error:5.2f} | {fixed}')


                       grand theift auto | -0.09 | grand theft auto
          belarus longitude and latitdue | -0.05 | belarus longitude and latitude
                       search for poeoms | -0.09 | search for poems
   large guacolmoi dip restaurtant price | -0.21 | large guacamole dip restaurant price
                 texas chainsaw mascurer | -0.28 | texas chainsaw massacre
                    royal trump subtitle | -0.07 | royal tramp subtitle
                florida fiberglass polls | -0.06 | florida fiberglass pools
                  how to make a calender | -0.07 | how to make a calendar
            university of south caroline | -0.05 | university of south carolina
            maureen mcdonald in virginia | -0.16 | maureen mcdonnell in virginia


## 5. Олтугеза
Объедините результат работы предыдущих пунктов в единый алгоритм исправления опечатки для запроса.

Примерный план:
1.	Для слов запроса генерируем список ближайших слов-кандидатов (для всех, даже словарных слов).
2.	Собираем список кандидатов-запросов (эвристически, чтобы не сделать экспоненциальное время выполнения)
3.	Для каждого кандидата считаем итоговый объединенный score на основе языковой модели и модели ошибок для данного кандидата (не обязательно сумма или произведение, можно объединение любой сложности).
4.	Выдаём гипотезу с наибольшим score.
5.	???
6.	Profit

In [90]:
from random import choice
from itertools import product

def correct(query: Query) -> Query:
    queries = []
    similar = {}
    for word in query.split(" "):
        similar[word] = sorted(
            find_similar_words(word),
            key=lambda x: get_error_probability(word, x),
            reverse=True
        )[:2]
    similar_queries = sorted(
        [" ".join(pr) for pr in product(*[similar[word] for word in query.split(" ")])],
        key=lambda x: get_probability(x),
        reverse=True
    )
    return similar_queries[0]

for original, fixed in queries_sample:
    predict = correct(original)
    verdict = '[ok]  ' if predict == fixed else '[fail]'
    sign = '==' if predict == fixed else '!='
    print(f'{verdict} {predict:>40s} {sign} {fixed}')

[ok]                           grand theft auto == grand theft auto
[ok]             belarus longitude and latitude == belarus longitude and latitude
[ok]                           search for poems == search for poems
[fail]       large giacomo dip restaurant price != large guacamole dip restaurant price
[fail]                    texas chainsaw maurer != texas chainsaw massacre
[fail]                     royal trump subtitle != royal tramp subtitle
[fail]                 florida fiberglass poles != florida fiberglass pools
[ok]                     how to make a calendar == how to make a calendar
[fail]             university of south caroline != university of south carolina
[fail]             maureen mcdonald in virginia != maureen mcdonnell in virginia


Итоговое качество меряем на примерах из `queries.tsv.gz`.

Для отладки проблем с качеством имеет смысл научится понимать на каком этапе теряется правильная гипотеза для каждого примера. Например, если правильное исправление есть в списке кандидатов (п. 2), но не выбирается как лучшая – стоит крутить языковую модель, модель ошибок и их объединение.

In [93]:
def check_corrector(queries: Queries, correct: Callable[[Query], Query], debug: bool):
    wrong, total = 0, 0
    progress = tqdm(queries[::100])
    debug_output = 0
    for original, fixed in progress:
        predict = correct(original)
        if predict != fixed:
            wrong += 1
            if debug:
                print(original)
                print(fixed)
                print(predict)
                print()
                debug_output += 1
                if debug_output == 10:
                    break
        total += 1
        progress.set_description(f'Wrong: {wrong} - {wrong/total*100:0.2f}%')
        
check_corrector(queries, correct, debug=False)


Все запросы проходят как-то очень долго. На половине запросов было стабильно 25.5%